### **Sentiment Analysis**

In [2]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 757de003-2cde-4eb6-aede-bc7f9188a67b)

In [1]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 3, Finished, Available, Finished)

In [5]:
#import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol('description')
        .setKind('SentimentAnalysis')
        .setOutputCol('response')
        .setErrorCol('error')
)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 7, Finished, Available, Finished)

In [6]:
#apply the model to our dataframe
result = model.transform(df)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 8, Finished, Available, Finished)

In [7]:
display(result)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5b9cb662-9c3b-4ee3-b56e-c6f8f29ed7c2)

In [8]:
#create sentiment column
from pyspark.sql.functions import col

sentiment_df = result.withColumn('sentiment', col('response.documents.sentiment'))

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b7b1f4f9-7f72-42da-a50a-18c081a55a83)

In [11]:
sentiment_df = sentiment_df.drop('error','response')
display(sentiment_df)

StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d8a2b078-4ed7-47fd-82bc-c25069b46f2b)

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print('Table Already Exists')

    sentiment_df.createOrReplaceTempView('view_sentiment_df')

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING view_df_sentiment_df source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND

                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished 
                   
                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
    
    """)


StatementMeta(, 4e25e117-3ee5-4266-adf0-42cd3c19cbcc, 14, Finished, Available, Finished)